# ResNet50 - Demo
This notebook is a demo on a very small example subset of the ISIC Dataset. The dataset includes five classes, with 4 images in each train class and 2 images in each validation class. This small dataset is intended to generate quick  results, and fit the storage limitations of a Github repo. Note that performance metrics will be very suboptimal with this small dataset. The batch size of the model is also changed from the original to better suit the dataset.
The ResNet50 model contained here is otherwise the same model I ran for this project using the entire ISIC dataset for classification, including pretrained Imagenet weights.

All necessary dependencies are installed via pip or imports by running the cells. To train and evaluate the model, follow these steps: 

1) Upload the dataset zip archive downloaded from the Github repo to Colab by clicking the Upload button under the Files tab on the left (it may take a little time to appear).

2) Simply run the below cells in order to train and evaluate the model.

The code was adapted with modifications from here:

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html 
License: BSD
Author: Sasank Chilamkurthy

In [2]:
# Installation of required packages 
!pip install timm==0.4.12 yacs==0.1.8 adversarial-robustness-toolbox
!pip install -U PyYAML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Resnet50 Pytorch

In [4]:
# Code adapted with modifications from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html 
# License: BSD
# Author: Sasank Chilamkurthy

# Imports

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import sklearn
from timm.utils import accuracy, AverageMeter

cudnn.benchmark = True
plt.ion()   # interactive mode

!unzip isic_example_data.zip

Archive:  isic_example_data.zip
   creating: isic_example_data/
   creating: isic_example_data/val/
   creating: isic_example_data/val/class5/
 extracting: isic_example_data/val/class5/ISIC_0024383.JPG  
 extracting: isic_example_data/val/class5/ISIC_0024382.JPG  
   creating: isic_example_data/val/class4/
 extracting: isic_example_data/val/class4/ISIC_0012090.JPG  
 extracting: isic_example_data/val/class4/ISIC_0010985.JPG  
   creating: isic_example_data/val/class3/
 extracting: isic_example_data/val/class3/ISIC_0024280.JPG  
 extracting: isic_example_data/val/class3/ISIC_0026090.JPG  
   creating: isic_example_data/val/class2/
 extracting: isic_example_data/val/class2/ISIC_0054090.JPG  
 extracting: isic_example_data/val/class2/ISIC_0046706.JPG  
   creating: isic_example_data/train/
   creating: isic_example_data/train/class4/
 extracting: isic_example_data/train/class4/ISIC_0001156.JPG  
 extracting: isic_example_data/train/class4/ISIC_0001144.JPG  
   creating: isic_example_data/

In [5]:
# Transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ]),
}

# Train on demo ISIC Dataset
data_dir = 'isic_example_data'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                  data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=2, # Batch size modified for this demo
              shuffle=True, num_workers=2)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=200):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    #del outputs

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                acc1, acc5 = accuracy(outputs,labels,topk=(1,5))

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            time_elapsed = time.time() - since
            print("This epoch takes ", time_elapsed, " sec")

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [10]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)

Epoch 0/19
----------
train Loss: 1.5824 Acc: 0.2083
This epoch takes  2.471799373626709
val Loss: 2.0800 Acc: 0.2000
This epoch takes  3.139185905456543

Epoch 1/19
----------
train Loss: 1.4868 Acc: 0.4167
This epoch takes  5.551735877990723
val Loss: 1.8090 Acc: 0.4000
This epoch takes  6.20919132232666

Epoch 2/19
----------
train Loss: 1.2947 Acc: 0.5000
This epoch takes  8.61259412765503
val Loss: 1.9876 Acc: 0.4000
This epoch takes  9.309433937072754

Epoch 3/19
----------
train Loss: 1.5486 Acc: 0.5417
This epoch takes  11.657067775726318
val Loss: 2.2074 Acc: 0.3000
This epoch takes  12.31187891960144

Epoch 4/19
----------
train Loss: 1.1488 Acc: 0.6667
This epoch takes  14.695588827133179
val Loss: 1.5939 Acc: 0.4000
This epoch takes  15.358679294586182

Epoch 5/19
----------
train Loss: 1.5915 Acc: 0.4583
This epoch takes  17.683232069015503
val Loss: 2.4140 Acc: 0.3000
This epoch takes  18.348335027694702

Epoch 6/19
----------
train Loss: 0.6884 Acc: 0.8750
This epoch tak